# Preamble

In [1]:
# Essentials
import os, sys, glob
import pandas as pd
import numpy as np
import nibabel as nib

# Stats
import scipy as sp
from scipy import stats
import statsmodels.api as sm
import pingouin as pg

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'

In [2]:
sys.path.append('/Users/lindenmp/Dropbox/Work/ResProjects/neurodev_long/code/func/')
from proj_environment import set_proj_env
from func import update_progress

In [3]:
exclude_str = 't1Exclude'
parcel_names, parcel_loc, drop_parcels, num_parcels, yeo_idx, yeo_labels = set_proj_env(exclude_str = exclude_str)

### Setup output directory

In [4]:
print(os.environ['MODELDIR'])
if not os.path.exists(os.environ['MODELDIR']): os.makedirs(os.environ['MODELDIR'])

/Users/lindenmp/Dropbox/Work/ResProjects/neurodev_long/analysis/normative/t1Exclude/schaefer_400


## Load train/test .csv and setup node .csv

In [5]:
df = pd.read_csv(os.path.join(os.environ['MODELDIR'], 'df_pheno.csv'))
df.set_index(['bblid', 'scanid', 'timepoint'], inplace = True)

print(df.shape)

(2307, 16)


In [6]:
df.head()

TotalNtimepoints  TotalNtimepoints_new  sex  race  \
bblid scanid timepoint                                                      
80010 2894   1                         2                     2    1     1   
      7211   2                         2                     2    1     1   
80179 2643   1                         1                     1    2     1   
80199 2637   1                         1                     1    1     5   
80208 3016   1                         1                     1    1     2   

                        ethnicity  scanageMonths  scanageYears  \
bblid scanid timepoint                                           
80010 2894   1                  2            262          21.8   
      7211   2                  2            293          24.4   
80179 2643   1                  2            255          21.2   
80199 2637   1                  1            245          20.4   
80208 3016   1                  2            246          20.5   

                        mprage_antsCT_vol_TBV  averageManualRating  \
bblid scanid timepoint                                               
80010 2894   1                   1.383110e+06                2.000   
      7211   2                   1.412140e+06                2.000   
80179 2643   1                   1.310120e+06                1.667   
80199 2637   1                   1.590630e+06                2.000   
80208 3016   1                   1.397398e+06                2.000   

                        dti32MeanRelRMS  Overall_Psychopathology     Mania  \
bblid scanid timepoint                                                       
80010 2894   1                 0.669736                 0.477260  0.234406   
      7211   2                 0.173847                -0.313878 -0.511101   
80179 2643   1                 0.321203                -0.355627 -0.489316   
80199 2637   1                 0.120190                 1.266311  2.008614   
80208 3016   1                 0.262002                 2.226531 -2.626595   

                        Depression  Psychosis_Positive  \
bblid scanid timepoint                                   
80010 2894   1            0.411101           -0.765745   
      7211   2            2.357006            0.169766   
80179 2643   1           -0.737088           -0.605224   
80199 2637   1            0.968194            0.970645   
80208 3016   1            0.526181           -0.372209   

                        Psychosis_NegativeDisorg  train_test  
bblid scanid timepoint                                        
80010 2894   1                          1.321310        True  
      7211   2                          0.555718        True  
80179 2643   1                          2.726989       False  
80199 2637   1                          1.385857       False  
80208 3016   1                         -1.461359       False

In [7]:
metrics = ('ct', 'vol')

In [8]:
# output dataframe
ct_labels = ['ct_' + str(i) for i in range(num_parcels)]
vol_labels = ['vol_' + str(i) for i in range(num_parcels)]

df_node = pd.DataFrame(index = df.index, columns = ct_labels + vol_labels)

print(df_node.shape)

(2307, 800)


## Load in data

In [9]:
CT = np.zeros((df.shape[0], num_parcels))

for (i, (index, row)) in enumerate(df.iterrows()):
    file_name = os.environ['CT_NAME_TMP'].replace("bblid", str(index[0]))
    file_name = file_name.replace("scanid", str(index[1]))
    full_path = glob.glob(os.path.join(os.environ['CTDIR'], file_name))
    if i == 0: print(full_path)    

    ct = np.loadtxt(full_path[0])
    CT[i,:] = ct
    
df_node.loc[:,ct_labels] = CT

['/Volumes/ResProjects_2TB/ResData/PNC/processedData/antsCorticalThickness/80010/20100218x2894/ct_schaefer400_17.txt']


In [11]:
# subject filter
subj_filt = np.zeros((df.shape[0],)).astype(bool)

In [12]:
VOL = np.zeros((df.shape[0], num_parcels))

for (i, (index, row)) in enumerate(df.iterrows()):
    file_name = os.environ['VOL_NAME_TMP'].replace("bblid", str(index[0]))
    file_name = file_name.replace("scanid", str(index[1]))
    full_path = glob.glob(os.path.join(os.environ['VOLDIR'], file_name))
    if i == 0: print(full_path)    

    img = nib.load(full_path[0])
    v = np.array(img.dataobj)
    v = v[v != 0]
    unique_elements, counts_elements = np.unique(v, return_counts=True)
    if len(unique_elements) == num_parcels:
        VOL[i,:] = counts_elements
    else:
        print(str(index) + '. Warning: not all parcels present')
        subj_filt[i] = True
    
df_node.loc[:,vol_labels] = VOL

['/Volumes/ResProjects_2TB/ResData/PNC/processedData/gm_vol_masks_native/80010/20100218x2894/Schaefer2018_400_17Networks_native_gm.nii.gz']
(80265, 3145, 1). Warning: not all parcels present
(87548, 3139, 1). Warning: not all parcels present


KeyboardInterrupt: 

In [10]:
df_node.head()

ct_0      ct_1      ct_2      ct_3      ct_4  \
bblid scanid timepoint                                                     
80010 2894   1          2.675105  2.589326  2.070390  2.110534  1.178882   
      7211   2          2.506903  2.417576  1.964256  1.983083  1.162326   
80179 2643   1          2.355527  2.321522  2.724572  1.804425  1.213590   
80199 2637   1          2.725121  2.576208  2.365333  1.794436  2.242248   
80208 3016   1          2.625381  1.889908  1.787339  1.278577  1.570609   

                            ct_5      ct_6      ct_7      ct_8      ct_9  ...  \
bblid scanid timepoint                                                    ...   
80010 2894   1          2.218669  1.691044  2.341620  1.601781  2.275076  ...   
      7211   2          2.260572  1.708321  2.245383  1.572678  2.192611  ...   
80179 2643   1          2.183063  1.814330  3.155051  2.251365  2.658216  ...   
80199 2637   1          2.753861  1.852478  2.226270  1.903049  2.956589  ...   
80208 3016   1          2.257836  1.659953  2.254210  1.650172  2.628392  ...   

                          ct_390    ct_391    ct_392    ct_393    ct_394  \
bblid scanid timepoint                                                     
80010 2894   1          2.707675  2.540489  2.248033  2.057684  2.085661   
      7211   2          2.497745  2.417623  2.334997  1.996006  2.056358   
80179 2643   1          2.172227  1.584825  1.667404  1.487523  2.900877   
80199 2637   1          3.011782  1.863701  1.970559  2.237102  2.386064   
80208 3016   1          2.473033  2.958341  1.543505  2.345227  2.157649   

                          ct_395    ct_396    ct_397    ct_398    ct_399  
bblid scanid timepoint                                                    
80010 2894   1          2.623213  2.289446  2.423470  2.379268  2.456141  
      7211   2          2.590235  2.369238  2.285462  2.305690  2.343947  
80179 2643   1          2.967420  1.945889  2.152096  1.852408  2.058653  
80199 2637   1          2.452310  2.165321  2.446768  2.077512  2.877450  
80208 3016   1          3.092755  2.373460  2.650476  3.520141  2.471672  

[5 rows x 400 columns]

Save out brain feature data before any nuisance regression

In [11]:
df_node.to_csv(os.path.join(os.environ['MODELDIR'], 'df_node_base.csv'))

### Nuisance regression

Regress out nuisance covariates. For cortical thickness, we regress out whole brain volume as well as average rating of scan quality from Roalf et al. 2018 NeuroImage.

We also used a mixed linear model for nuisance regression (instead of a simple OLS) to account for dependency in some of the data

In [12]:
nuis = ['mprage_antsCT_vol_TBV', 'averageManualRating']
df_nuis = df.loc[:,nuis]
df_nuis = sm.add_constant(df_nuis)

for i, col in enumerate(df_node.columns):
    update_progress(i/df_node.shape[1])
    mdl = sm.MixedLM(df_node.loc[:,col], df_nuis, groups = df.reset_index()['bblid']).fit()
    y_pred = mdl.predict(df_nuis)
    df_node.loc[:,col] = df_node.loc[:,col] - y_pred
update_progress(1)

 Progress: [####################] 100.0%


In [13]:
df_node.head()

ct_0      ct_1      ct_2      ct_3      ct_4  \
bblid scanid timepoint                                                     
80010 2894   1         -0.184296  0.143393 -0.277037  0.057025 -0.499676   
      7211   2         -0.362687 -0.032789 -0.391947 -0.074330 -0.519712   
80179 2643   1         -0.402244 -0.035664  0.495949 -0.110394 -0.326379   
80199 2637   1         -0.207117  0.098591 -0.044834 -0.286983  0.538807   
80208 3016   1         -0.239034 -0.558206 -0.564408 -0.776853 -0.109661   

                            ct_5      ct_6      ct_7      ct_8      ct_9  ...  \
bblid scanid timepoint                                                    ...   
80010 2894   1         -0.406283 -0.442588 -0.527129 -0.252785 -0.414380  ...   
      7211   2         -0.380744 -0.424646 -0.632248 -0.283834 -0.511827  ...   
80179 2643   1         -0.299956 -0.245366  0.351272  0.480006  0.048934  ...   
80199 2637   1          0.011936 -0.276404 -0.705967  0.034578  0.160032  ...   
80208 3016   1         -0.375170 -0.473352 -0.618910 -0.205352 -0.068438  ...   

                          ct_390    ct_391    ct_392    ct_393    ct_394  \
bblid scanid timepoint                                                     
80010 2894   1         -0.237792  0.132205  0.354469 -0.149294 -0.216552   
      7211   2         -0.466274  0.002461  0.430897 -0.224749 -0.259872   
80179 2643   1         -0.581851 -0.690593 -0.152824 -0.555507  0.673992   
80199 2637   1         -0.066305 -0.593752  0.001682 -0.068362 -0.016356   
80208 3016   1         -0.481565  0.546672 -0.355244  0.131468 -0.151463   

                          ct_395    ct_396    ct_397    ct_398    ct_399  
bblid scanid timepoint                                                    
80010 2894   1         -0.336122 -0.037489 -0.096955 -0.373399 -0.120761  
      7211   2         -0.371715  0.035606 -0.252835 -0.455576 -0.249343  
80179 2643   1          0.139265 -0.311152 -0.188599 -0.804838 -0.366086  
80199 2637   1         -0.525720 -0.209487 -0.201414 -0.736628  0.183396  
80208 3016   1          0.132132  0.043229  0.121255  0.763241 -0.113296  

[5 rows x 400 columns]

## Save out

In [14]:
df_node.to_csv(os.path.join(os.environ['MODELDIR'], 'df_node_clean.csv'))

In [11]:
df_node = pd.read_csv(os.path.join(os.environ['MODELDIR'], 'df_node_clean.csv'))

In [13]:
df_node.set_index(['bblid','scanid'], inplace = True)

In [15]:
df_node.head()

timepoint      ct_0      ct_1      ct_2      ct_3      ct_4  \
bblid scanid                                                                
80010 2894            1 -0.184296  0.143393 -0.277037  0.057025 -0.499676   
      7211            2 -0.362687 -0.032789 -0.391947 -0.074330 -0.519712   
80179 2643            1 -0.402244 -0.035664  0.495949 -0.110394 -0.326379   
80199 2637            1 -0.207117  0.098591 -0.044834 -0.286983  0.538807   
80208 3016            1 -0.239034 -0.558206 -0.564408 -0.776853 -0.109661   

                  ct_5      ct_6      ct_7      ct_8  ...    ct_390    ct_391  \
bblid scanid                                          ...                       
80010 2894   -0.406283 -0.442588 -0.527129 -0.252785  ... -0.237792  0.132205   
      7211   -0.380744 -0.424646 -0.632248 -0.283834  ... -0.466274  0.002461   
80179 2643   -0.299956 -0.245366  0.351272  0.480006  ... -0.581851 -0.690593   
80199 2637    0.011936 -0.276404 -0.705967  0.034578  ... -0.066305 -0.593752   
80208 3016   -0.375170 -0.473352 -0.618910 -0.205352  ... -0.481565  0.546672   

                ct_392    ct_393    ct_394    ct_395    ct_396    ct_397  \
bblid scanid                                                               
80010 2894    0.354469 -0.149294 -0.216552 -0.336122 -0.037489 -0.096955   
      7211    0.430897 -0.224749 -0.259872 -0.371715  0.035606 -0.252835   
80179 2643   -0.152824 -0.555507  0.673992  0.139265 -0.311152 -0.188599   
80199 2637    0.001682 -0.068362 -0.016356 -0.525720 -0.209487 -0.201414   
80208 3016   -0.355244  0.131468 -0.151463  0.132132  0.043229  0.121255   

                ct_398    ct_399  
bblid scanid                      
80010 2894   -0.373399 -0.120761  
      7211   -0.455576 -0.249343  
80179 2643   -0.804838 -0.366086  
80199 2637   -0.736628  0.183396  
80208 3016    0.763241 -0.113296  

[5 rows x 401 columns]

In [24]:
from func import summarise_network

df_system = summarise_network(df_node, parcel_loc, yeo_idx, metrics = ('ct',), method = 'mean')

df_system = pd.concat((df_node.filter(regex = 'ct', axis = 1).mean(axis = 1).rename('ct'),df_system),axis = 1)

In [25]:
df_system.to_csv('/Users/lindenmp/Dropbox/Work/git/python_cookbook/data/df_long_system.csv')

In [26]:
df.to_csv('/Users/lindenmp/Dropbox/Work/git/python_cookbook/data/df_long_pheno.csv')